<a href="https://colab.research.google.com/github/Guthikonda-Akshaya/GanForge/blob/assignment-3/a3q1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Task1
import torch
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.utils import to_categorical    # required by the question

# 1. Load the Iris dataset
iris = load_iris()
X = iris.data  # Features
y = iris.target  # Labels

# 2. Train-test split (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# 3. Standardize features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# 4. One-hot encode target labels using to_categorical()
y_train_encoded = to_categorical(y_train, num_classes=3)
y_test_encoded = to_categorical(y_test, num_classes=3)

# 5. Convert to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train_encoded, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test_encoded, dtype=torch.float32)

# Check tensor shapes
print("X_train:", X_train_tensor.shape)
print("y_train:", y_train_tensor.shape)

X_train: torch.Size([120, 4])
y_train: torch.Size([120, 3])


In [ ]:
#Task2
import torch.nn as nn
import torch.nn.functional as F

# Define the Neural Network class
class IrisNet(nn.Module):
    def __init__(self):
        super(IrisNet, self).__init__()

        # Input layer to hidden layer (4 -> 8)
        self.fc1 = nn.Linear(4, 8)

        # Hidden layer to output layer (8 -> 3)
        self.fc2 = nn.Linear(8, 3)

    def forward(self, x):
        # Hidden layer with ReLU activation
        x = F.relu(self.fc1(x))

        # Output layer with Softmax activation
        x = F.softmax(self.fc2(x), dim=1)  # softmax along the class dimension
        return x

# Instantiate the model
model = IrisNet()

# Print model architecture
print(model)


IrisNet(
  (fc1): Linear(in_features=4, out_features=8, bias=True)
  (fc2): Linear(in_features=8, out_features=3, bias=True)
)


In [ ]:
#task3
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader

# Define batch size and epochs
batch_size = 5
epochs = 100

# 1. Create TensorDataset and DataLoader
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

# 2. Loss function: categorical cross-entropy (use nn.CrossEntropyLoss if labels are class indices,
# but we are using one-hot labels, so we use nn.BCELoss with softmax outputs)
loss_function = nn.BCELoss()  # since softmax + one-hot => use BCELoss (not CrossEntropyLoss)

# 3. Optimizer: Adam
optimizer = optim.Adam(model.parameters(), lr=0.01)

# 4. Training loop
for epoch in range(epochs):
    total_loss = 0
    for batch_X, batch_y in train_loader:
        # Forward pass
        outputs = model(batch_X)

        # Compute loss
        loss = loss_function(outputs, batch_y)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    # Print loss every 10 epochs
    if (epoch+1) % 10 == 0:
        print(f"Epoch [{epoch+1}/{epochs}], Loss: {total_loss:.4f}")


Epoch [10/100], Loss: 1.5419
Epoch [20/100], Loss: 1.1415
Epoch [30/100], Loss: 0.8097
Epoch [40/100], Loss: 0.7421
Epoch [50/100], Loss: 0.7157
Epoch [60/100], Loss: 0.6253
Epoch [70/100], Loss: 0.5547
Epoch [80/100], Loss: 0.5081
Epoch [90/100], Loss: 0.5449
Epoch [100/100], Loss: 0.5474


In [ ]:
#Task4
# Disable gradient calculation for evaluation
with torch.no_grad():
    # Forward pass on test data
    outputs = model(X_test_tensor)  # shape: [30, 3]

    # Predicted class is the index with the highest probability
    predicted_classes = torch.argmax(outputs, dim=1)  # shape: [30]

    # Actual class (convert one-hot encoded test labels to class indices)
    actual_classes = torch.argmax(y_test_tensor, dim=1)

    # Calculate accuracy
    correct = (predicted_classes == actual_classes).sum().item()
    total = actual_classes.size(0)
    accuracy = correct / total * 100

    print(f"Test Accuracy: {accuracy:.2f}%")


Test Accuracy: 96.67%
